In [1]:
import scrapy
import requests
from scrapy.http import TextResponse 

In [2]:
url  = "http://job.incruit.com/entry/searchjob.asp?rgn2=11&jobty=1&occ1=150&page=1"

In [3]:
req = requests.get(url)
response = TextResponse(req.url, body = req.text, encoding = 'utf-8')

In [4]:
response

<200 http://job.incruit.com/entry/searchjob.asp?rgn2=11&jobty=1&occ1=150&page=1>

In [5]:
links = response.xpath('//*[@id="incruit_contents"]/div[3]/div[1]/div[2]/table/tbody/tr/td[1]/div/span[1]/a/@href').extract()

In [6]:
links = response.xpath('//*[@id="incruit_contents"]/div[3]/div[1]/div[2]/table/tbody/tr[1]/td[1]/div/span[1]/a/@href').extract()

In [7]:
links

['http://job.incruit.com/entry/jobpost.asp?job=2007200001091']

In [10]:
!rm -rf incruit

!scrapy startproject incruit

New Scrapy project 'incruit', using template directory '/Users/hyunjoon/opt/anaconda3/lib/python3.7/site-packages/scrapy/templates/project', created in:
    /Users/hyunjoon/Downloads/incruit

You can start your first spider with:
    cd incruit
    scrapy genspider example example.com


In [11]:
!tree incruit

incruit
├── incruit
│   ├── __init__.py
│   ├── __pycache__
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       └── __pycache__
└── scrapy.cfg

4 directories, 7 files


In [12]:
%%writefile incruit/incruit/items.py
import scrapy

class IncruitItem(scrapy.Item):

    title = scrapy.Field()
    content = scrapy.Field()
    employment_form = scrapy.Field()
    status = scrapy.Field()

Overwriting incruit/incruit/items.py


In [13]:
%%writefile incruit/incruit/spiders/spider.py
import scrapy
import requests
from incruit.items import IncruitItem
from scrapy.http import TextResponse 

class Spider(scrapy.Spider):
    name = "Week_Incruit"
    allow_domain = ["https://www.incruit.com/"]
    start_urls = ["http://job.incruit.com/entry/searchjob.asp?ct=12&ty=1&cd=1"]
    
    def parse(self, response): 
        for i in range(1,10):
            url = url  = "http://job.incruit.com/entry/searchjob.asp?jobty=4&jobty=1&occ1=150&rgn2=11&page={}".format(i)
            req = requests.get(url)
            response = TextResponse(req.url, body = req.text, encoding = 'utf-8')
            links = response.xpath('//*[@id="incruit_contents"]/div[3]/div[1]/div[2]/table/tbody/tr/td[1]/div/span[1]/a/@href').extract()
            for link in links:
                yield scrapy.Request(link, callback = self.page_content)

    def page_content(self, response):
        item = IncruitItem()
        item["title"] = response.xpath('//*[@id="content"]/div[2]/h1/span/text()').extract()
        item["content"] = response.xpath('//*[@id="content"]/div[2]/h1/strong/text()').extract()
        item["employment_form"] = response.xpath('//*[@id="content"]/div[3]/div[1]/div[2]/div[3]/dl[1]/dd/div/div[1]/em/text()').extract()
        item["status"] = response.xpath('//*[@id="content"]/div[3]/div[1]/div[2]/div[1]/dl[1]/dd/div/div/text()').extract()
        yield item

Writing incruit/incruit/spiders/spider.py


In [14]:
%%writefile run.sh
cd incruit
scrapy crawl Week_Incruit -o Week_Incruit.csv

Overwriting run.sh


In [15]:
!chmod +x run.sh

In [16]:
! ./run.sh

2020-07-27 21:11:02 [scrapy.utils.log] INFO: Scrapy 2.2.0 started (bot: incruit)
2020-07-27 21:11:02 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.6 (default, Jan  8 2020, 13:42:34) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Darwin-18.7.0-x86_64-i386-64bit
2020-07-27 21:11:02 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-07-27 21:11:02 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'incruit',
 'NEWSPIDER_MODULE': 'incruit.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['incruit.spiders']}
2020-07-27 21:11:02 [scrapy.extensions.telnet] INFO: Telnet Password: c5803a9850c34122
2020-07-27 21:11:02 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',

2020-07-27 21:11:04 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2006050000841>
{'content': ['현대글로비스 전문인력 상시채용(부문: 신사업/연구소)'],
 'employment_form': [],
 'status': ['\t\t\t\t\t\t\t2001년 02월 23일\t\t\t\t\t\t'],
 'title': ['현대글로비스']}
2020-07-27 21:11:04 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://job.incruit.com/entry/jobpost.asp?job=2006180001011> (referer: http://job.incruit.com/entry/searchjob.asp?ct=12&ty=1&cd=1)
2020-07-27 21:11:05 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2006180001011>
{'content': ['2020년 신입석박사 상시채용[정보보안기술 개발]'],
 'employment_form': ['정규직'],
 'status': ['\t\t\t\t통신·모바일 > 보안 외 2건\t\t\t\t', '\t\t\t', '\t\t\t\t', '\t\t\t'],
 'title': ['케이티']}
2020-07-27 21:11:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://job.incruit.com/entry/jobpost.asp?job=2007270000411> (referer: http://job.incruit.com/entry/searchjob.asp?ct=12&ty=1&cd=1)
2020-07-27 21:11:05 [scrapy.c

2020-07-27 21:11:06 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2002190004682>
{'content': ['백엔드, 프론트엔드 개발자 (서버개발자는 AWS구축경험자)'],
 'employment_form': ['정규직'],
 'status': ['\t\t\t\t웹프로그래머 > 백엔드\t\t\t', '\t\t\t\t', '\t\t\t'],
 'title': ['포씨아']}
2020-07-27 21:11:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://job.incruit.com/entry/jobpost.asp?job=2001300000655> (referer: http://job.incruit.com/entry/searchjob.asp?ct=12&ty=1&cd=1)
2020-07-27 21:11:06 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2001300000655>
{'content': ['스크린골프 소프트웨어 개발 신규 인력 모집'],
 'employment_form': ['정규직'],
 'status': ['\t\t\t\t웹기획·웹마케팅·PM > 웹기획\t\t\t', '\t\t\t\t', '\t\t\t'],
 'title': ['지에스비즈']}
2020-07-27 21:11:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://job.incruit.com/entry/jobpost.asp?job=2002210002281> (referer: http://job.incruit.com/entry/searchjob.asp?ct=12&ty=1&cd=1)
2020-07-27 21:11:06 [scrapy.

2020-07-27 21:11:09 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2001280001682>
{'content': ['통신공사 현장 신입/경력 정규직, 인턴 채용'],
 'employment_form': ['정규직', '인턴'],
 'status': ['\t\t\t\t통신·모바일 > 통신공사, 통신설계 외 1건\t\t\t\t',
            '\t\t\t',
            '\t\t\t\t',
            '\t\t\t'],
 'title': ['01-technet']}
2020-07-27 21:11:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://job.incruit.com/entry/jobpost.asp?job=2004290002674> (referer: http://job.incruit.com/entry/searchjob.asp?ct=12&ty=1&cd=1)
2020-07-27 21:11:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://job.incruit.com/entry/jobpost.asp?job=1909270001965> (referer: http://job.incruit.com/entry/searchjob.asp?ct=12&ty=1&cd=1)
2020-07-27 21:11:09 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2004290002674>
{'content': ['(주)듀코젠 3d 모델링 디자이너, VR/AR 콘텐츠 개발자, VR/AR콘텐츠 기획 담당자 모집'],
 'employment_form': ['정규직'],
 'status': ['\t\t\t\t광고·

2020-07-27 21:11:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://job.incruit.com/entry/jobpost.asp?job=2005270001118> (referer: http://job.incruit.com/entry/searchjob.asp?ct=12&ty=1&cd=1)
2020-07-27 21:11:12 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2005270001118>
{'content': ['[주식회사 클라모스] React/NodeJS 풀스텍 개발자 채용'],
 'employment_form': ['정규직'],
 'status': ['\t\t\t\t소프트웨어·하드웨어 > 소프트웨어 외 3건\t\t\t\t',
            '\t\t\t',
            '\t\t\t\t',
            '\t\t\t'],
 'title': ['클라모스']}
2020-07-27 21:11:13 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://job.incruit.com/entry/jobpost.asp?job=2006090003638> (referer: http://job.incruit.com/entry/searchjob.asp?ct=12&ty=1&cd=1)
2020-07-27 21:11:13 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://job.incruit.com/entry/jobpost.asp?job=2005120003698> (referer: http://job.incruit.com/entry/searchjob.asp?ct=12&ty=1&cd=1)
2020-07-27 21:11:13 [scrapy.core.engine] DEBUG: Crawled 

...

2020-07-27 21:12:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://job.incruit.com/entry/jobpost.asp?job=2007270002711> (referer: http://job.incruit.com/entry/searchjob.asp?ct=12&ty=1&cd=1)
2020-07-27 21:12:33 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2007270002711>
{'content': ['[카트 서울신설법인] 카트:드리프트 월드빌더 모집'],
 'employment_form': ['정규직'],
 'status': ['\t\t\t\t게임(Game) > 게임디자인 외 2건\t\t\t\t',
            '\t\t\t',
            '\t\t\t\t',
            '\t\t\t'],
 'title': ['넥슨코리아']}
2020-07-27 21:12:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://job.incruit.com/entry/jobpost.asp?job=2007170003817> (referer: http://job.incruit.com/entry/searchjob.asp?ct=12&ty=1&cd=1)
2020-07-27 21:12:33 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2007170003817>
{'content': ['JSP, Servlet 외 신입/경력 정규직, 병역특례 채용'],
 'employment_form': ['정규직', '병역특례'],
 'status': ['\t\t\t\t웹프로그래머 > JSP, Servlet 외 2건

2020-07-27 21:12:35 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2007220001781>
{'content': ['[마케팅] 바이럴 마케팅 신입사원 모집'],
 'employment_form': ['인턴'],
 'status': ['\t\t\t\t컨텐츠·사이트운영 > 커뮤니티운영, 컨텐츠기획·운영 외 1건\t\t\t\t',
            '\t\t\t',
            '\t\t\t\t',
            '\t\t\t'],
 'title': ['에듀윌']}
2020-07-27 21:12:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://job.incruit.com/entry/jobpost.asp?job=2006090001114> (referer: http://job.incruit.com/entry/searchjob.asp?ct=12&ty=1&cd=1)
2020-07-27 21:12:35 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2006090001114>
{'content': ['관제사업 분야 신입/ 경력직 채용 (보안관제, 보안운영, 보안진단, 보안장비 운영, CERT)'],
 'employment_form': ['정규직', '계약직'],
 'status': ['\t\t\t\t네트워크·서버·보안 > 정보보안, 보안관제\t\t\t', '\t\t\t\t', '\t\t\t'],
 'title': ['SK인포섹']}
2020-07-27 21:12:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://job.incruit.com/entry/jobpost.asp?job=2007220001960>

2020-07-27 21:12:38 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2006180001002>
{'content': ['2020년 신입석박사 상시채용[융합보안기술 개발]'],
 'employment_form': ['정규직'],
 'status': ['\t\t\t\t네트워크·서버·보안 > 정보보안 외 2건\t\t\t\t',
            '\t\t\t',
            '\t\t\t\t',
            '\t\t\t'],
 'title': ['케이티']}
2020-07-27 21:12:38 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2006170002429>
{'content': ['부문별 신입 및 경력 상시 모집(기술지원)'],
 'employment_form': ['정규직'],
 'status': ['\t\t\t\t설치·수리·정비·A/S > 엘리베이터·에스컬레이터, 기계설치·정비 외 1건\t\t\t\t',
            '\t\t\t',
            '\t\t\t\t',
            '\t\t\t'],
 'title': ['현대엘리베이터']}
2020-07-27 21:12:38 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://job.incruit.com/entry/jobpost.asp?job=2007170003144> (referer: http://job.incruit.com/entry/searchjob.asp?ct=12&ty=1&cd=1)
2020-07-27 21:12:38 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://job.incruit.com/entry/jo

2020-07-27 21:12:41 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2006300002545>
{'content': ['[커뮤니케이션실] PR 매니저 (인턴)'],
 'employment_form': ['인턴'],
 'status': ['\t\t\t\t홍보·PR·전시·컨벤션 > 홍보, 홍보기획 외 1건\t\t\t\t',
            '\t\t\t',
            '\t\t\t\t',
            '\t\t\t'],
 'title': ['야놀자']}
2020-07-27 21:12:41 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2007220000204>
{'content': ['[NHN] 2020 경영지원 인턴 채용'],
 'employment_form': [],
 'status': ['\t\t\t\t\t\t\t2013년 08월 01일\t\t\t\t\t\t'],
 'title': ['NHN']}
2020-07-27 21:12:41 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2006180000393>
{'content': ['2020 상반기 해외 석박사 R&D 채용[전력전자]'],
 'employment_form': ['정규직'],
 'status': ['\t\t\t\t연구원·연구개발 > 전기·전자 외 2건\t\t\t\t',
            '\t\t\t',
            '\t\t\t\t',
            '\t\t\t'],
 'title': ['LG이노텍']}
2020-07-27 21:12:41 [scrapy.core.engine] DEBUG: 

2020-07-27 21:12:44 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2006180000886>
{'content': ['2020년 신입석박사 상시채용[실감미디어기술 개발]'],
 'employment_form': ['정규직'],
 'status': ['\t\t\t\t광고·CG·시각디자인 > 3D디자인·모델링 외 2건\t\t\t\t',
            '\t\t\t',
            '\t\t\t\t',
            '\t\t\t'],
 'title': ['케이티']}
2020-07-27 21:12:44 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp?job=2007160000841>
{'content': ['2020년도 4직급(대졸, 경력) 직원 채용공고'],
 'employment_form': ['정규직'],
 'status': ['\t\t\t\t치료·의료기사 > 산업위생관리기사 외 3건\t\t\t\t',
            '\t\t\t',
            '\t\t\t\t',
            '\t\t\t'],
 'title': ['한국중부발전']}
2020-07-27 21:12:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://job.incruit.com/entry/jobpost.asp?job=2006150000987> (referer: http://job.incruit.com/entry/searchjob.asp?ct=12&ty=1&cd=1)
2020-07-27 21:12:44 [scrapy.core.scraper] DEBUG: Scraped from <200 http://job.incruit.com/entry/jobpost.asp

In [17]:
!ls incruit/

Week_Incruit.csv incruit          scrapy.cfg


In [18]:
df = pd.read_csv("incruit/Week_Incruit.csv")
df['status'] = df['status'].apply(lambda x: str(x).replace('\t',''))


In [20]:
df['status'] = df['status'].apply(lambda x: str(x).replace('\t',''))

In [22]:
df

,content,employment_form,status,title
0,2021년도 한국은행 신입 종합기획직원(G5) 채용 안내,정규직,"조사·분석·통계 > 사회조사분석, 통계·SAS 외 3건,,,",한국은행
1,대우조선해양 회계분야 경력/신입사원 수시채용,정규직,"전산·IT기술지원 > IT기술지원 외 3건,,,",대우조선해양
2,2020년 신입석박사 상시채용[영상분석기술 개발],정규직,"ERP·시스템분석·설계 > 딥러닝 외 2건,,,",케이티
3,2020년 신입석박사 상시채용[자연어처리기술 개발],정규직,"ERP·시스템분석·설계 > 딥러닝 외 2건,,,",케이티
4,2020년 신입석박사 상시채용[SW아키텍트],정규직,"데이터베이스·DBA > 데이터아키텍처 외 3건,,,",케이티
...,...,...,...,...
539,2020년도 하반기 청년인턴 채용,NaN,1976년 06월 21일,한국수출입은행
540,[틱톡] 각 부문별 채용,"정규직,인턴","컨텐츠·사이트운영 > 컨텐츠에디터, 컨텐츠기획·운영 외 33건,,,",바이트댄스
541,[KT] 2020년 KT 신입석박사 상시채용(인재DB등록),정규직,"네트워크·서버·보안 > 시스템·네트워크보안, 네트워크구축·운영·관리 외 9건,,,",케이티
542,2020년 신입석박사 상시채용[AI 음성기술 개발],정규직,"빅데이터·AI(인공지능) > 인공지능(AI) 외 2건,,,",케이티
